In [196]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score
import xlsxwriter

In [197]:
stocks = pd.read_csv("../Project 1/sp_500_stocks.csv")

In [198]:
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'
base_url = "https://sandbox.iexapis.com/stable/stock/"
col_names = ["Ticker", "Price", "One-Year Price Return"]

In [199]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
            yield lst[i:i+n]

In [200]:
symbol_groups = list(chunks(stocks["Ticker"], 100))
symbol_strings = []

for x in symbol_groups:
    symbol_strings.append(','.join(x))

In [201]:
final_dataframe = pd.DataFrame(columns = col_names)

In [202]:
for symbol_string in symbol_strings:
    batch_api_call_url = f"{base_url}market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        if symbol == "WLTW":
            continue
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]["price"],
                    data[symbol]["stats"]["year1ChangePercent"],
                ],
                index = col_names
            ),
            ignore_index = True
        )

In [203]:
final_dataframe.sort_values("One-Year Price Return", ascending=False, inplace=True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True, drop=True)

In [204]:
#we wrap the functionality of calculating the number of shares into a function
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter your portfolio size :")
    
    try:
        portfolio_size = int(portfolio_size)
    except ValueError:
        print("The value entered was not a number!\n Please enter a number")
        portfolio_size = input("Enter your portfolio size :")
        portfolio_size = int(portfolio_size)

portfolio_input()

Enter your portfolio size :10000000


In [205]:
position_size = portfolio_size/final_dataframe.shape[0]

def append_number_of_shares_to_buy(df, ps):
    a = np.asarray(df["Price"])
    df.loc[:, "Number of shares to buy"] = np.floor(ps*(1/a))

In [206]:
append_number_of_shares_to_buy(final_dataframe, position_size)

In [207]:
hqm_columns = [
    "Ticker",
    "Price",
    "Number of shares to buy",
    "One-Year Price Return",
    "One-Year Return Percentile",
    "Six-Month Price Return",
    "Six-Month Return Percentile",
    "Three-Month Price Return",
    "Three-Month Return Percentile",
    "One-Month Price Return",
    "One-Month Return Percentile",
    "HQM Score"
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f"{base_url}market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        if symbol == "WLTW":
            continue
        hqm_dataframe = hqm_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]["price"],
                    'N/A',
                    data[symbol]["stats"]["year1ChangePercent"],
                    'N/A',
                    data[symbol]["stats"]["year1ChangePercent"],
                    'N/A',
                    data[symbol]["stats"]["year1ChangePercent"],
                    'N/A',
                    data[symbol]["stats"]["year1ChangePercent"],
                    'N/A',
                    'N/A'
                ],
                index = hqm_columns
            ),
            ignore_index = True
        )

In [208]:
hqm_dataframe.fillna(0,inplace=True)
time_periods = [
    "One-Year",
    "Six-Month",
    "Three-Month",
    "One-Month"
]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f"{time_period} Price Return"
        percentile_col = f"{time_period} Return Percentile"
        hqm_dataframe.loc[row, percentile_col] = (score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col]))/100

In [209]:
hqm_dataframe

,Ticker,Price,Number of shares to buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,139.19,N/A,0.064626,0.361111,0.064626,0.361111,0.064626,0.361111,0.064626,0.361111,N/A
1,AAL,19.57,N/A,0.009455,0.269841,0.009455,0.269841,0.009455,0.269841,0.009455,0.269841,N/A
2,AAP,228.40,N/A,0.510130,0.922619,0.510130,0.922619,0.510130,0.922619,0.510130,0.922619,N/A
3,AAPL,180.49,N/A,0.256917,0.684524,0.256917,0.684524,0.256917,0.684524,0.256917,0.684524,N/A
4,ABBV,144.91,N/A,0.441493,0.886905,0.441493,0.886905,0.441493,0.886905,0.441493,0.886905,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
499,YUM,124.00,N/A,0.168009,0.555556,0.168009,0.555556,0.168009,0.555556,0.168009,0.555556,N/A
500,ZBH,122.55,N/A,-0.281069,0.025794,-0.281069,0.025794,-0.281069,0.025794,-0.281069,0.025794,N/A
501,ZBRA,453.83,N/A,-0.111923,0.125,-0.111923,0.125,-0.111923,0.125,-0.111923,0.125,N/A
502,ZION,77.41,N/A,0.507568,0.918651,0.507568,0.918651,0.507568,0.918651,0.507568,0.918651,N/A


In [210]:
from statistics import mean

In [211]:
for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f"{time_period} Return Percentile"])
    hqm_dataframe.loc[row, "HQM Score"] = mean(momentum_percentiles)

hqm_dataframe

,Ticker,Price,Number of shares to buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,139.19,N/A,0.064626,0.361111,0.064626,0.361111,0.064626,0.361111,0.064626,0.361111,0.361111
1,AAL,19.57,N/A,0.009455,0.269841,0.009455,0.269841,0.009455,0.269841,0.009455,0.269841,0.269841
2,AAP,228.40,N/A,0.510130,0.922619,0.510130,0.922619,0.510130,0.922619,0.510130,0.922619,0.922619
3,AAPL,180.49,N/A,0.256917,0.684524,0.256917,0.684524,0.256917,0.684524,0.256917,0.684524,0.684524
4,ABBV,144.91,N/A,0.441493,0.886905,0.441493,0.886905,0.441493,0.886905,0.441493,0.886905,0.886905
...,...,...,...,...,...,...,...,...,...,...,...,...
499,YUM,124.00,N/A,0.168009,0.555556,0.168009,0.555556,0.168009,0.555556,0.168009,0.555556,0.555556
500,ZBH,122.55,N/A,-0.281069,0.025794,-0.281069,0.025794,-0.281069,0.025794,-0.281069,0.025794,0.025794
501,ZBRA,453.83,N/A,-0.111923,0.125,-0.111923,0.125,-0.111923,0.125,-0.111923,0.125,0.125
502,ZION,77.41,N/A,0.507568,0.918651,0.507568,0.918651,0.507568,0.918651,0.507568,0.918651,0.918651


In [212]:
hqm_dataframe.sort_values("HQM Score", ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(drop=True, inplace=True)

In [213]:
portfolio_input()

Enter your portfolio size :10000000


In [214]:
position_size = float(portfolio_size/hqm_dataframe.shape[0])
append_number_of_shares_to_buy(hqm_dataframe, position_size)

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [215]:
hqm_dataframe

,Ticker,Price,Number of shares to buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,LB,82.38,2427.0,2.367720,1.0,2.367720,1.0,2.367720,1.0,2.367720,1.0,1.0
1,DVN,53.08,3767.0,1.758028,0.998016,1.758028,0.998016,1.758028,0.998016,1.758028,0.998016,0.998016
2,MRO,21.26,9407.0,1.412719,0.996032,1.412719,0.996032,1.412719,0.996032,1.412719,0.996032,0.996032
3,NUE,126.63,1579.0,1.156729,0.994048,1.156729,0.994048,1.156729,0.994048,1.156729,0.994048,0.994048
4,COP,90.98,2198.0,1.003512,0.992063,1.003512,0.992063,1.003512,0.992063,1.003512,0.992063,0.992063
5,EOG,114.50,1746.0,0.971751,0.990079,0.971751,0.990079,0.971751,0.990079,0.971751,0.990079,0.990079
6,FANG,132.26,1512.0,0.928259,0.988095,0.928259,0.988095,0.928259,0.988095,0.928259,0.988095,0.988095
7,FTNT,337.26,593.0,0.891540,0.986111,0.891540,0.986111,0.891540,0.986111,0.891540,0.986111,0.986111
8,MCHP,75.98,2632.0,0.841090,0.984127,0.841090,0.984127,0.841090,0.984127,0.841090,0.984127,0.984127
9,PXD,220.82,905.0,0.781179,0.982143,0.781179,0.982143,0.781179,0.982143,0.781179,0.982143,0.982143


In [216]:
writer = pd.ExcelWriter("momentum strategy.xlsx", engine = "xlsxwriter")
hqm_dataframe.to_excel(writer, sheet_name="Momentum Strategy", index = False)

In [217]:
# creating different formats for strings, numbers and monetary values
background_color = "0a0a23"
font_color = "#ffffff"

string_format = writer.book.add_format(
    {
        "font_color": font_color,
        "bg_color": background_color,
        "border": 1
    }
)

dollar_format = writer.book.add_format(
    {
        "num_format": "$0.00",
        "font_color": font_color,
        "bg_color": background_color,
        "border": 1
    }
)

number_format = writer.book.add_format(
    {
        "num_format": "0",
        "font_color": font_color,
        "bg_color": background_color,
        "border": 1
    }
)

percent_format = writer.book.add_format(
    {
        "num_format": "0.0%",
        "font_color": font_color,
        "bg_color": background_color,
        "border": 1
    }
)

In [218]:
# incorporating the various formats into the sheet and then finally saving it
column_formats = {
    "A" : ["Ticker", string_format],
    "B" : ["Price", dollar_format],
    "C" : ["Number of shares to buy", number_format],
    "D" : ["One-Year Price Return", percent_format],
    "E" : ["One-Year Return Percentile", percent_format],
    "F" : ["Six-Month Price Return", percent_format],
    "G" : ["Six-Month Return Percentile", percent_format],
    "H" : ["Three-Month Price Return", percent_format],
    "I" : ["Three-Month Return Percentile", percent_format],
    "J" : ["One-Month Price Return", percent_format],
    "K" : ["One-Month Return Percentile", percent_format],
    "L" : ["HQM Score", percent_format],
}

for column in column_formats.keys():
    writer.sheets["Momentum Strategy"].set_column(f"{column}:{column}", 25, column_formats[column][1])
    writer.sheets["Momentum Strategy"].write(f"{column}1", column_formats[column][0], column_formats[column][1])

writer.save()